# Inception Model

In [3]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Dense, concatenate, AveragePooling2D, GlobalAveragePooling2D, Flatten
from keras.layers.merge import concatenate
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler

def inception_block(input_layer, f1, f2_in, f2_out, f3_in, f3_out, f4_out):
    # 1x1 conv
    conv1 = Conv2D(f1, (1,1), padding='same', activation='relu')(input_layer)
    # 3x3 conv
    conv3 = Conv2D(f2_in, (1,1), padding='same', activation='relu')(input_layer)
    conv3 = Conv2D(f2_out, (3,3), padding='same', activation='relu')(conv3)
    # 5x5 conv
    conv5 = Conv2D(f3_in, (1,1), padding='same', activation='relu')(input_layer)
    conv5 = Conv2D(f3_out, (5,5), padding='same', activation='relu')(conv5)
    # 3x3 max pooling
    pool = MaxPooling2D((3,3), strides=(1,1), padding='same')(input_layer) 
    pool = Conv2D(f4_out, (1,1), padding='same', activation='relu')(pool)
    # concatenate filters
    layer_out = concatenate([conv1, conv3, conv5, pool], axis=-1)
    return layer_out

Input =  Input(shape = (224, 224, 3))

conv_1 = Conv2D(64, (7, 7), padding = 'same', strides = (2, 2), activation = 'relu')(Input)
maxpool_1 = MaxPooling2D((3, 3), padding = 'same', strides = (2, 2))(conv_1)

conv_2 = Conv2D(64, (1 ,1), padding = 'same', strides = (1, 1), activation = 'relu')(maxpool_1)
conv_3 = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu')(conv_2)
maxpool_2 = MaxPooling2D((3, 3), padding='same', strides=(2, 2))(conv_3)

#inception block 1
inception_1 = inception_block(maxpool_2, 64, 96, 128, 16, 32, 32)

#inception block 2
inception_2 = inception_block(inception_1, 128, 128, 192, 32, 96, 64)

maxpool_3 = MaxPooling2D((3, 3), padding = 'same', strides = (2, 2))(inception_2)

#inception block 3
inception_3 = inception_block(maxpool_3, 192, 96, 208, 16, 48, 64)

avgpool_1 = AveragePooling2D((5, 5), strides = 3)(inception_3)
conv_4 = Conv2D(128, (1, 1), padding = 'same', activation = 'relu')(avgpool_1)
flatten_1 = Flatten()(conv_4)

dense_1 = Dense(1024, activation = 'relu')(flatten_1)
dropout_1 = Dropout(0.7)(dense_1)
dense_2 = Dense(10, activation = 'softmax')(dropout_1)

#inception block 4
inception_4 = inception_block(inception_3, 60, 112, 224, 24, 64, 64)

#inception block 5
inception_5 = inception_block(inception_4, 128, 128, 256, 24, 64, 64)

#inception block 6
inception_6 = inception_block(inception_5, 112, 144, 288, 32, 64, 64)

avgpool_2 = AveragePooling2D((5, 5), strides = 3)(inception_6)
conv_5 = Conv2D(128, (1, 1), padding = 'same', activation = 'relu')(avgpool_2)
flatten_2 = Flatten()(conv_5)
dense_3 = Dense(1024, activation = 'relu')(flatten_2)
dropout_2 = Dropout(0.7)(dense_3)
dense_4 = Dense(10, activation = 'softmax')(dropout_2)

#inception block 7
inception_7 = inception_block(inception_6, 256, 160, 320, 32, 128, 128)

maxpool_4 = MaxPooling2D((3, 3), padding = 'same', strides = (2, 2))(inception_7)

inception_8 = inception_block(maxpool_4, 256, 160, 320, 32, 128, 128)

inception_9 = inception_block(inception_8, 384, 192, 384, 48, 128,128)

global_avg_pool = GlobalAveragePooling2D()(inception_9)

dropout_3 = Dropout(0.4)(global_avg_pool)

dense_5 = Dense(10, activation = 'softmax')(dropout_3)

model = Model(Input, [dense_5, dense_2, dense_4])


In [2]:
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 56, 56, 64)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 56, 56, 64)   4160        max_pooling2d[0][0]              
______________________________________________________________________________________________

This is the model presented in the paper by __Christian Szegedy__, __Wei Liu__.

Papers Link: [Going Deeper with Convolutions](https://arxiv.org/abs/1409.4842)